# Create a catalog order

## Step 1. Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, rasterio, geojson, pathlib
import pandas as pd

### 2. Configure credentials

Run the cell below to create a `credentials.json` file in a directory named `.up42` in your home folder.

In [ ]:
# Defines the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Prints the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
1. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
1. Save the `credentials.json` file.

### 3. Authenticate

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

# Use region if you're logging with the NSG service
# up42.authenticate(cfg_file=credentials_file_path, region="sa")

## Step 2. Get access

### 1. Select a catalog collection

You can browse all archive collections in our glossary

In [ ]:
archive_collections = up42.CollectionType.ARCHIVE
sort_by_name = up42.CollectionSorting.name
data_collections = up42.ProductGlossary.get_collections(
    collection_type=archive_collections, sort_by=sort_by_name
)
pd.DataFrame(
    {"collection": c.name, "product": d.name}
    for c in data_collections
    for d in c.data_products
)

Choose a data product.

In [ ]:
# The sample uses Pléiades catalog — Analytic

# Retrieve a list of geospatial collections
data_collections = up42.ProductGlossary.get_collections(
    collection_type=archive_collections, sort_by=sort_by_name
)

# Iterate over collections and select Pléiades ("phr")
collection = next(c for c in data_collections if c.name == "phr")

# Find the Analytic data product
data_product = next(d for d in collection.data_products if d.name.endswith("analytic"))

# Retrieve the host
host = next(p for p in collection.providers if p.is_host)
collection.name, data_product.name, data_product.id, host.title

### 2. Request access

If you want to order the chosen collection for the first time, you need to [request access to it](https://docs.up42.com/getting-started/restrictions#request-access).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

### 3. Accept the EULA

If you want to order the chosen collection for the first time, you need to accept its [end-user license agreement (EULA)](https://docs.up42.com/getting-started/account/eulas).

## Step 3. Find and preview data

### 1. Search the catalog

Search the host to find a full scene that fits your requirements.

In [ ]:
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [13.369713, 52.452327],
            [13.369713, 52.470760],
            [13.339159, 52.470760],
            [13.339159, 52.452327],
            [13.369713, 52.452327],
        ]
    ],
}

scenes = list(
    host.search(
        collections=[collection.name],
        intersects=geometry,
        start_date="2022-06-01",
        end_date="2022-12-31",
        query={"cloudCoverage": {"LT": 20}},
    )
)

Output a dataframe with full scenes that match the specified parameters.

In [ ]:
pd.DataFrame(scenes)
# scenes # Uncomment to output data class instances instead

### 2. Preview data

Some data hosts provide free, low-resolution quicklooks that you can preview before creating an order. Note that some quicklooks might not fully match the scene they represent.

You can visualize the quicklooks using Jupyter display components.

In [ ]:
from IPython.display import Image

# Download quicklook to quicklooks directory
scene = scenes[0]
quicklook_path = scene.quicklook.download("./quicklooks/")
Image(quicklook_path)

Since the quicklook image is not georeferenced, you might need to georeference it. 

In [ ]:
import shapely as shp

# Define a function to georeference quicklook


def georeference(
    src_path: str, dst_path: str, bounds: tuple[float, float, float, float]
):
    with rasterio.open(src_path) as src:
        data = src.read()
        transform = rasterio.transform.from_bounds(
            *bounds, data.shape[2], data.shape[1]
        )

        with rasterio.open(
            dst_path,
            "w",
            driver=src.driver,
            height=src.height,
            width=src.width,
            count=src.count,
            dtype=src.dtypes[0],
            crs="EPSG:4326",
            transform=transform,
        ) as dst:
            dst.write(data)


# Georefence the quicklooks
out_path = f"./quicklooks/georeferenced_quicklook_{scene.id}.png"
georeference(
    quicklook_path,
    out_path,
    shp.geometry.shape(scene.geometry).bounds,
)

Open the `quicklooks` folder. You will find the following files added by the cell above:

- The original quicklook
- A metadata file needed for georeferencing
- The georeferenced quicklook with the `georeferenced` prefix

[Visualize](https://docs.up42.com/help/visualize) the georeferenced quicklook in a third-party GIS software.

## Step 4. Create an order

### 1. Fill out an order form

Get detailed information about the parameters needed to create an order for the chosen data product.

In [ ]:
data_product.schema

Use the required request body schema format for the chosen data product.

In [ ]:
from datetime import date

features = geojson.FeatureCollection(features=[geojson.Feature(geometry=geometry)])
order_template = up42.BatchOrderTemplate(
    data_product_id=data_product.id,
    display_name=f"{data_product.name}-{date.today()}",
    features=features,
    params={"id": scene.id},
    tags=["sdk", data_product.name],
)

### 2. Get a cost estimate

Get a cost estimate before placing the order.

In [ ]:
order_template.estimate

The response returns the overall credit amount that will be deducted from your credit balance if you decide to proceed with the ordering.

### 3. Create an order

Create an order with the requested parameters.

Credits will be deducted upon successful completion of the created catalog order. The transaction can't be reversed.

In [ ]:
order_references = order_template.place()
order = order_references[0].order
order

## Step 5. Monitor orders

Check the [status](https://docs.up42.com/developers/api-catalog#order-statuses) of your order. To learn about the timeframe of catalog orders, see [Asset delivery time](https://docs.up42.com/data/archive/delivery).

In [ ]:
order.status

You can also track the order status until the order is completed.

In [ ]:
order.track(report_time=150)

## Step 6. Download assets

The following statuses mean you can download assets from storage:
- `BEING_FULFILLED`: Some order assets might have been delivered.
- `FULFILLED`: All order assets have been delivered.